In [47]:
!pip install sentencepiece


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import numpy as np
from nltk.tokenize.treebank import TreebankWordDetokenizer
from transformers import pipeline, AutoTokenizer
from nltk.tokenize import wordpunct_tokenize
mask_filler = pipeline("fill-mask", 'distilbert-base-uncased')

In [49]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

#### Здесь описываются параметры и задается входное предложение:  

In [50]:
SENT_NUM = 10
TEMPERATURE = 0.05
TOP_K = 5
input_sent = "After your workout, remember to focus on maintaining a good water balance."
#input_sent = "The latest tech news about the world's best (and sometimes worst) hardware, apps, and much more."
#input_sent = "Remember to drink enough water to restore and maintain your body's hydration after your cardio training."

In [51]:
WORDS_TO_REPLACE = len(wordpunct_tokenize(input_sent))//3
input_sent = input_sent.lower()
curr_res = input_sent
detokenizer = TreebankWordDetokenizer()
predicted_sentences = []
for _ in range(SENT_NUM):
    for _ in range(WORDS_TO_REPLACE):
        words = wordpunct_tokenize(curr_res)
        word_to_replace = np.random.randint(len(words))
        words[word_to_replace] = tokenizer.mask_token
        sequence = detokenizer.detokenize(words)
        prediction = mask_filler(sequence, top_k=1)[0]
        curr_res = prediction['sequence']
        if prediction == words[word_to_replace-1]:
            del words[word_to_replace]
            curr_res = detokenizer.detokenize(words)
        # если предсказываем токен = предыдущему слову -> убираем его:
        # c вероятностью TEMPERATURE или пока предложение остается тем же возьмем predict на позиции [1 TOP_K), а не самый вероятный
        if np.random.rand() < TEMPERATURE:
            curr_res = mask_filler(sequence, top_k=TOP_K)[np.random.randint(1,TOP_K)]['sequence']
        while curr_res == input_sent:
            curr_res = mask_filler(sequence, top_k=TOP_K)[np.random.randint(1,TOP_K)]['sequence']
    predicted_sentences.append(curr_res)
    curr_res = input_sent

In [53]:
predicted_sentences

['enjoy your workout, caution : focus on maintaining a good water balance.',
 'after your workout you remember to focus on maintaining reasonably good water flow.',
 'enjoy your workout, remember to focus on maintaining a good physical balance!',
 'after your workout, remember to concentrate, maintaining a good water balance.',
 'after your shower you remember to focus on maintaining a good water balance.',
 'after your workout you remember : focus on getting a good water balance.',
 'after your workout, try to focus on maintaining fairly stable water balance.',
 'after your workout, try to focus on maintaining a good water supply.',
 'enjoy your bath, remember to focus on maintaining very good water balance.',
 'after your shower, remember to focus on maintaining a good physical balance.']